![LogoUC3M](https://upload.wikimedia.org/wikipedia/commons/thumb/a/a6/Acr%C3%B3nimo_y_nombre_de_la_UC3M.svg/320px-Acr%C3%B3nimo_y_nombre_de_la_UC3M.svg.png) 

*Alonso Rios Guerra - 100495821 | Guillermo Sancho González - 100495991*


# *__Aprendizaje automático P1: Predicción del abandono de empleados__*

## *__Introducción__*

En esta práctica tenemos como objetivo desarrollar diferentes métodos de aprendizaje automático para predecir el abandono de los trabajadores de una empresa.

Primero de todo empezaremos leyendo los datos que nos proporciona la empresa.

In [10]:
import numpy as np
import pandas as pd

data_path = 'attrition_availabledata_10.csv.gz'

data = pd.read_csv(data_path, compression='gzip', sep = ',')


## *__EDA Simplificado__*

Un EDA es una análisis exploratorio de datos, para organizar los datos, entender su contenido, entender cual son las variables más relevantes y cómo se relacionan unas con otras, determinar qué hacer con los datos faltantes y con los datos atípicos, y finalmente extraer conclusiones acerca de todo este análisis.

Para hacer un eda debemos responder a distintas preguntas:

-   ¿Cuántas instancias y atributos hay?

-   ¿Qué tipo de atributos hay (numéricos o categóricos)? Esto se hace para verificar si hay características categóricas que deben ser codificadas (como variables dummy o one-hot encoding). Comprobar si hay variables categóricas con alta cardenalidad.

-   ¿Qué atributos tienen valores faltantes y cuántos?

-   ¿Existen columnas constantes o ID?

-   ¿Es un problema de clasificación o regresión (variable de respuesta) y? En caso de clasificación, ¿las clases están desbalanceadas?

A continuación le damos respuesta:



-   ¿Cuántas instancias y atributos hay?

In [12]:
print('La forma de la tabla es:')
print('===============================')
print(data.shape)

La forma de la tabla es:
(2940, 31)


El dataset contiene 2940 instancias, 30 atributos y 1 etiqueta (Attrition).

-   ¿Qué tipo de atributos hay (numéricos o categóricos)? Esto se hace para verificar si hay características categóricas que deben ser codificadas (como variables dummy o one-hot encoding). Comprobar si hay variables categóricas con alta cardenalidad.

In [24]:
print('Los tipos de atributos son:')
print('================================')
data.info()

Los tipos de atributos son:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2940 entries, 0 to 2939
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   hrs                      2940 non-null   float64
 1   absences                 2940 non-null   int64  
 2   JobInvolvement           2940 non-null   int64  
 3   PerformanceRating        2940 non-null   int64  
 4   EnvironmentSatisfaction  2925 non-null   float64
 5   JobSatisfaction          2928 non-null   float64
 6   WorkLifeBalance          2911 non-null   float64
 7   Age                      2940 non-null   int64  
 8   BusinessTravel           2940 non-null   object 
 9   Department               2940 non-null   object 
 10  DistanceFromHome         2940 non-null   int64  
 11  Education                2940 non-null   int64  
 12  EducationField           2940 non-null   object 
 13  EmployeeCount            2940 non-null   int64  
 

In [18]:

print('Los tipos de atributos son:')
print('================================')
data.info()

print()

print('Cuantos valores faltan por atributo:')
print('======================================')
print(data.isnull().sum())

print()

print('Comprobar si la clase está desbalanceada:')
print('======================================')
print(data['Attrition'].value_counts())
print()
print(data['Attrition'].value_counts() / data['Attrition'].count())

Los tipos de atributos son:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2940 entries, 0 to 2939
Data columns (total 31 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   hrs                      2940 non-null   float64
 1   absences                 2940 non-null   int64  
 2   JobInvolvement           2940 non-null   int64  
 3   PerformanceRating        2940 non-null   int64  
 4   EnvironmentSatisfaction  2925 non-null   float64
 5   JobSatisfaction          2928 non-null   float64
 6   WorkLifeBalance          2911 non-null   float64
 7   Age                      2940 non-null   int64  
 8   BusinessTravel           2940 non-null   object 
 9   Department               2940 non-null   object 
 10  DistanceFromHome         2940 non-null   int64  
 11  Education                2940 non-null   int64  
 12  EducationField           2940 non-null   object 
 13  EmployeeCount            2940 non-null   int64  
 

La etiqueta es Attrition, que puede tener los valores Yes o No, en el que las clases estan bastante desbalanceadas: 83.88% No, 16.12% Yes

Hay 8 variables categóricas que hay que codificar.

Hay 5 variables que les faltan valores (EnviromentSatisfaction, JobSatisfaction, WorkLifeBalance, NumCompaniesWorked y TotalWorkingYears)


In [6]:
# 1. Comprobar columnas constantes
constantes = [col for col in data.columns if data[col].nunique() == 1]
print("Columnas constantes:", constantes)

# 2. Comprobar columnas ID
columnas_id = [col for col in data.columns if data[col].nunique() == len(data)]
print("Columnas ID:", columnas_id)


Columnas constantes: ['EmployeeCount', 'Over18', 'StandardHours']
Columnas ID: ['EmployeeID']


Como hay 3 columnas que no varían y otra que es un ID, se les elimina del dataframe porque no aportan ninguna información útil


In [7]:
data = data.drop(columns= constantes + columnas_id)

Despues, vemos todos los valores distintos que tiene las variables categóricas y sus percentiles

In [13]:
# Filtrar las columnas de tipo 'object' (categóricas)
categoricos = data.select_dtypes(include=['object']).columns

numericos = data.select_dtypes(include=['float64']).columns

# Mostrar las categorías y su porcentaje para cada columna categórica
for col in categoricos:
    print(f"Categorías para la columna '{col}':")
    print(data[col].value_counts(normalize=True) * 100)  # Multiplicamos por 100 para obtener el porcentaje
    print("\n")

Categorías para la columna 'BusinessTravel':
BusinessTravel
Travel_Rarely        70.918367
Travel_Frequently    18.775510
Non-Travel           10.306122
Name: proportion, dtype: float64


Categorías para la columna 'Department':
Department
Research & Development    65.782313
Sales                     29.965986
Human Resources            4.251701
Name: proportion, dtype: float64


Categorías para la columna 'EducationField':
EducationField
Life Sciences       41.054422
Medical             31.972789
Marketing           10.612245
Technical Degree     9.251701
Other                5.238095
Human Resources      1.870748
Name: proportion, dtype: float64


Categorías para la columna 'Gender':
Gender
Male      61.156463
Female    38.843537
Name: proportion, dtype: float64


Categorías para la columna 'JobRole':
JobRole
Sales Executive              21.666667
Research Scientist           20.816327
Laboratory Technician        17.857143
Manufacturing Director        9.829932
Healthcare Representa

Para poder operar con estos datos categóricos hay que codificarlos como One-Hot-Encoding, o de manera ordinal si un atributo tiene algún tipo de cardinalidad.
En este caso consideramos que ninguna de los 8 atributos son cardinales por lo que los codificaremos todos con One-Hot-Encoding

Primero se dividrirán las categorias en input y output

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

X_data = data.drop('Attrition', axis=1)

Y_data = data['Attrition']

preprocessor = ColumnTransformer(
    transformers = [
                    ('categorical', OneHotEncoder(handle_unknown='ignore'), categoricos),
                    ('numerical', MinMaxScaler(), numericos)
                   ],
                   remainder='passthrough')

In [25]:
from sklearn import neighbors

preprocessor.fit(X_data)
X = preprocessor.transform(X_data)

# Notice that now we have 7 columnos
print(X.shape)
print()

# Notice that now the type of the data matrix is numpy, which can already be used by sklearn
print(type(X))
print()
"""
clf = neighbors.KNeighborsClassifier()

# Now, we train (fit) the method on the (X,y) dataset
clf.fit(X_data, Y_data)
"""


ValueError: A given column is not a column of the dataframe